### Registration

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'..\CCM', nargout=0)
eng.addpath(r'..\CCM\util', nargout=0)

In [3]:
import sys
sys.path.append('..')

from CCM import framework as methods
from CCM import assessment as assess

import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

# groups = range(10) # for all 10 groups
# steps = range(0, 2001, 100) # for all 20 steps

groups = range(1)
steps = range(1000, 1001)

# read the fixed image and mask
fixed_location = '../Data/NS/'
fixed = sitk.ReadImage(os.path.join(fixed_location, 'fixed.mha'))
mask = sitk.ReadImage(os.path.join(fixed_location, 'mask.mha'))

for group in groups:
    print('group', group)
    output_location = f'../Data/NS/NS_Group_{group}/'

    for step in steps:
            
        print(step)
        
        # read moving image
        iter_output_location = os.path.join(output_location, "Step_%04d" % step)
        moving = sitk.ReadImage(os.path.join(iter_output_location, 'moving.mha'))
        gt_field = sitk.ReadImage(os.path.join(iter_output_location, 'gt_field.mha'))

        print('\rProposed ...', end='')
        reg = methods.Registration(fixed=fixed, moving=moving, matlab_eng=eng)
        reg.PerformCCMRegistration(
            nlevel=[2,1], nmigr=[3,3], niter=[200,200], step=1, sigma=16,
            feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=150, feature_detector_ratio=[4,2], feature_detector_num=[1000,1000],
            feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0, 3, 5, 7, 9], feature_descriptor_samplenum=[1, 10, 15, 20, 25], feature_descriptor_dmax=5.5, feature_descriptor_dmin=8,
            feature_match_ratio=1, feature_match_crosscheck=True,
            route_merge=True, route_check=True, route_stay=0.01,
            ccm_k=[50,50], ccm_e=[0.5,0.5], ccm_j=[0.5,0.5], ccm_t=[0.5,0.5], ccm_z=[0.5,0.5])
        reg.OutputRegistrationResult(iter_output_location+'/ccm_')
        print('\rProposed, done.')
        print(assess.Assessment(registration=reg, mask=mask, gt_field=gt_field).AssessRegistration())

group 0
1000
Proposed, done.
{'mse': [44.25567510825987, 132.08372020633666, 112675], 'jac': [1.000082502274798, 0.12653486659871896, 112675], 'tre': [-1, -1, -1], 'err': [0.9558484923760736, 1.817069925647859, 112675], 'SDlogJ': 0.8171705771788795, 'dice': [], 'hdd': [], 'time': [7.5957818031311035]}
